In [1]:

import os,sys
sys.path.append(os.path.abspath(".")) 
sys.path.append(os.path.abspath("BIPIA")) 
sys.path.append(os.path.abspath("aside")) # to access aside.experiments
from aside.model import *
from aside.model_api import *
import torch
from transformers import AutoTokenizer,AutoModelForCausalLM
import json
import numpy as np
from datasets import load_dataset
from tqdm import tqdm
from collections import defaultdict,Counter
from utils.utils import *
from utils.model_utils import load_model
from copy import deepcopy
from utils.plot_utils import *
from functools import partial
torch.set_grad_enabled(False)
seed_all()
from BIPIA.bipia.data import AutoPIABuilder
from BIPIA.bipia.metrics import BipiaEvalFactory
from vllm import SamplingParams
from constants import *
from accelerate import PartialState
PartialState() 

/export/home2/weijie210/miniconda3/envs/ipi_hw/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/export/home2/weijie210/miniconda3/envs/ipi_hw/compiler_compat/ld: warning: librt.so.1, needed by /cm/shared/apps/cuda12.8/toolkit/12.8.1/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/export/home2/weijie210/miniconda3/envs/ipi_hw/compiler_compat/ld: warning: libpthread.so.0, needed by /cm/shared/apps/cuda12.8/toolkit/12.8.1/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/export/home2/weijie210/miniconda3/envs/ipi_hw/compiler_compat/ld: warning: libstdc++.so.6, needed by /cm/shared/apps/cuda12.8/toolkit/12.8.1/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/export/home2/weijie210/miniconda3/envs/ipi_hw/compiler_compat/ld: warning: libm.so.6, needed by /cm/shared/apps/cuda12.8/toolkit/12.8.1/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/export/home2/weijie210/minic

INFO 10-13 15:21:10 [__init__.py:216] Automatically detected platform cuda.


Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

In [2]:
device = 'cuda:0'
torch_dtype = torch.bfloat16
max_ctx_len = 32768
# model_path = 'Qwen/Qwen3-8B'
model_path = QWEN_PATH
# model_path = 'Qwen3-8B_sft_ASIDE_ToolAndAlpaca'
# model_path = os.path.join(MODEL_DIR,model_path)

use_vllm=True
model,tokenizer,is_aside,init_fn = load_model(model_path,use_vllm=use_vllm,dtype=torch_dtype,vllm_kwargs = {'gpu_memory_utilization':0.8,'enable_chunked_prefill':False},max_ctx_len=max_ctx_len)



INFO 10-13 15:21:13 [utils.py:328] non-default args: {'max_model_len': 32768, 'gpu_memory_utilization': 0.8, 'disable_log_stats': True, 'enable_chunked_prefill': False, 'model': 'Qwen/Qwen3-8B'}
INFO 10-13 15:21:25 [__init__.py:742] Resolved architecture: Qwen3ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 10-13 15:21:25 [__init__.py:1815] Using max model len 32768
INFO 10-13 15:21:29 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 10-13 15:21:30 [__init__.py:2974] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 10-13 15:21:37 [__init__.py:216] Automatically detected platform cuda.
(EngineCore_DP0 pid=1564795) INFO 10-13 15:21:38 [core.py:654] Waiting for init message from front-end.
(EngineCore_DP0 pid=1564795) INFO 10-13 15:21:38 [core.py:76] Initializing a V1 LLM engine (v0.10.2) with config: model='Qwen/Qwen3-8B', speculative_config=None, tokenizer='Qwen/Qwen3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_form

[W1013 15:21:41.089168683 ProcessGroupNCCL.cpp:981] Warning: TORCH_NCCL_AVOID_RECORD_STREAMS is the default now, this environment variable is thus deprecated. (function operator())


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
(EngineCore_DP0 pid=1564795) INFO 10-13 15:21:41 [parallel_state.py:1165] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0, EP rank 0
(EngineCore_DP0 pid=1564795) WARNING 10-13 15:21:41 [topk_topp_sampler.py:69] FlashInfer is not available. Falling back to the PyTorch-native implementation of top-p & top-k sampling. For the best performance, please install FlashInfer.
(EngineCore_DP0 pid=1564795) INFO 10-13 15:21:41 [gpu_model_ru

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  20% Completed | 1/5 [00:00<00:02,  1.95it/s]
Loading safetensors checkpoint shards:  40% Completed | 2/5 [00:01<00:02,  1.05it/s]
Loading safetensors checkpoint shards:  60% Completed | 3/5 [00:03<00:02,  1.25s/it]
Loading safetensors checkpoint shards:  80% Completed | 4/5 [00:04<00:01,  1.39s/it]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:06<00:00,  1.47s/it]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:06<00:00,  1.32s/it]
(EngineCore_DP0 pid=1564795) 


(EngineCore_DP0 pid=1564795) INFO 10-13 15:21:50 [default_loader.py:268] Loading weights took 6.66 seconds
(EngineCore_DP0 pid=1564795) INFO 10-13 15:21:50 [gpu_model_runner.py:2392] Model loading took 15.2683 GiB and 8.033123 seconds
(EngineCore_DP0 pid=1564795) INFO 10-13 15:21:58 [backends.py:539] Using cache directory: /export/home2/weijie210/.cache/vllm/torch_compile_cache/77df5711bc/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=1564795) INFO 10-13 15:21:58 [backends.py:550] Dynamo bytecode transform time: 7.68 s
(EngineCore_DP0 pid=1564795) INFO 10-13 15:22:03 [backends.py:161] Directly load the compiled graph(s) for dynamic shape from the cache, took 4.160 s
(EngineCore_DP0 pid=1564795) INFO 10-13 15:22:04 [monitor.py:34] torch.compile takes 7.68 s in total
(EngineCore_DP0 pid=1564795) INFO 10-13 15:22:06 [gpu_worker.py:298] Available KV cache memory: 46.66 GiB
(EngineCore_DP0 pid=1564795) INFO 10-13 15:22:06 [kv_cache_utils.py:864] GPU KV cache size: 339,792 to

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|█| 67/67 [00:03<0


(EngineCore_DP0 pid=1564795) INFO 10-13 15:22:10 [gpu_model_runner.py:3118] Graph capturing finished in 4 secs, took 0.62 GiB
(EngineCore_DP0 pid=1564795) INFO 10-13 15:22:10 [gpu_worker.py:391] Free memory on device (78.76/79.25 GiB) on startup. Desired GPU memory utilization is (0.8, 63.4 GiB). Actual usage is 15.27 GiB for weight, 1.45 GiB for peak activation, 0.02 GiB for non-torch memory, and 0.62 GiB for CUDAGraph memory. Replace gpu_memory_utilization config with `--kv-cache-memory=49284854784` to fit into requested memory, or `--kv-cache-memory=65780986880` to fully utilize gpu memory. Current kv cache memory in use is 50104841216 bytes.
(EngineCore_DP0 pid=1564795) INFO 10-13 15:22:10 [core.py:218] init engine (profile, create kv cache, warmup model) took 20.14 seconds
INFO 10-13 15:22:12 [llm.py:295] Supported_tasks: ['generate']
INFO 10-13 15:22:12 [__init__.py:36] No IOProcessor plugins requested by the model


In [3]:
from pathlib import Path
import jsonlines

def evaluate_scenario(
    model,
    tokenizer,
    scenario_name,
    attack="text",
):
    if attack == "text":
        attack_path = "BIPIA/benchmark/text_attack_test.json"
    elif attack == "code":
        attack_path = "BIPIA/benchmark/code_attack_test.json"
    else:
        raise ValueError(f"Invalid attack type: {attack}")

    scenario_to_path = {
        "email": "BIPIA/benchmark/email/test.jsonl",
        "code": "BIPIA/benchmark/code/test.jsonl",
        "qa": "BIPIA/benchmark/qa/test.jsonl",
        "abstract": "BIPIA/benchmark/abstract/test.jsonl",
        "table": "BIPIA/benchmark/table/test.jsonl",
    }
    pia_builder = AutoPIABuilder.from_name(scenario_name)(2023)
    pia_samples = pia_builder(
        scenario_to_path[scenario_name],
        attack_path,
        enable_stealth=False,
    )

    # Filter samples for specific attacks (which do no require LLM evaluation)
    df = pia_samples[
        pia_samples.attack_name.str.startswith("Emoji Substitution")
        | pia_samples.attack_name.str.startswith("Language Translation")
        | pia_samples.attack_name.str.contains("Marketing & Advertising-1")
        | pia_samples.attack_name.str.contains("Scams & Fraud-1")
        | pia_samples.attack_name.str.contains(
            "Bringing Down Hosts and Servers (Denial of Service)"
        )
        | pia_samples.attack_name.str.contains("Compromising Computers")
        | pia_samples.attack_name.str.contains(
            "Encrypting Documents and Demanding Ransom (Ransomware)"
        )
        | pia_samples.attack_name.str.contains("Corrupting an Operating System")
        | pia_samples.attack_name.str.contains("Blocking Internet Connection")
        | pia_samples.attack_name.str.contains("Introduce System Fingerprinting")
        | pia_samples.attack_name.str.contains("Screen Scraping")
        | pia_samples.attack_name.str.contains("Keylogging")
        | pia_samples.attack_name.str.contains("Traffic Analysis")
        | pia_samples.attack_name.str.contains("Data Eavesdropping")
    ]

    # Prepare prompts
    all_prompts = []
    def format_bipia_prompt(instr,data, tokenizer):
        has_input = 'message.role == "input"' in tokenizer.chat_template
        if has_input:
            msg = [{'role':'user','content':instr},{'role':'input','content':data}]
        else:
            msg = [{'role':'user','content':instr+'\n\n'+data}]
        return msg
    gen_kwargs = SamplingParams(temperature=0., max_tokens=1024,stop=[tokenizer.eos_token])
    
    gpt_config_file = "BIPIA/config/gpt35.yaml"  # Make sure this exists
    # Define attacks to evaluate
    attacks = np.array(df.attack_name.unique())
    

    for i in range(len(df)):
        user_prompt, input_prompt = pia_builder.construct_prompt(df.iloc[i]) # edit the construct_aside_prompt to 
        all_prompts.append(tool_prompt_format(format_bipia_prompt(user_prompt,input_prompt,tokenizer),None,tokenizer))

    # Generate responses using the model
    responses = vllm_generate(model,all_prompts,gen_kwargs,use_tqdm=True)
    # Prepare output for evaluation
    out = []
    if "question" not in df.columns:
        df["question"] = ""
    for attack_name, task_name, target, message, position, response in zip(
        df["attack_name"],
        df["task_name"],
        df["ideal"],
        df["question"],
        df["position"],
        responses,
    ):
        out.append(
            {
                "attack_name": attack_name,
                "task_name": task_name,
                "response": response,
                "message": message,
                "target": target,
                "position": position,
            }
        )

    # Save responses
    output_path = Path(f"BIPIA/output/{scenario_name}_responses.jsonl")
    output_path.parent.mkdir(exist_ok=True, parents=True)
    with jsonlines.open(output_path, "w") as writer:
        writer.write_all(out)

    # Evaluate responses
    gpt_config_file = "BIPIA/config/gpt35.yaml"  # Make sure this exists
    # Define attacks to evaluate
    attacks = np.array(df.attack_name.unique())

    evaluator = BipiaEvalFactory(
        gpt_config=gpt_config_file,
        activate_attacks=attacks,
    )
    

    asrs = evaluator.add_batch(
        predictions=responses,
        references=df["ideal"],
        attacks=df["attack_name"],
        tasks=df["task_name"],
    )

    avg_asr = np.mean(asrs)
    print(f"The average ASR for {scenario_name} scenario is: {avg_asr}")

    return df, asrs, avg_asr
    
def evaluate_bipia(
    model,
    tokenizer,
    scenarios=["email", "code", "table", "abstract", "qa"],
    attacks=["text", "code"],
):
    results = defaultdict(dict)
    total_count = 0
    for attack_type in attacks:
        print(f"\nEvaluating BIPIA {attack_type} attacks:")

        # Use only code scenario for code attacks, and other scenarios for text attacks
        if attack_type == "code":
            attack_scenarios = ["code"]
        else:
            attack_scenarios = [s for s in scenarios if s != "code"]

        print(f"Using scenarios {attack_scenarios} for {attack_type} attacks")
        scenario_asr = []
        for scenario in attack_scenarios:
            print(
                f"\nEvaluating {scenario} scenario with attack: {attack_type}"
            )
            _, asrs, avg_asr = evaluate_scenario(
                model,
                tokenizer,
                scenario,
                attack_type,
            )
            scenario_asr.append(avg_asr)
            print(
                f"Attack: {attack_type}, Scenario: {scenario}, ASR: {avg_asr:.4f}, Count: {len(asrs)}"
            )
            total_count += len(asrs)
            results[attack_type][scenario] = np.round(avg_asr,3)
            # Calculate average ASR across all scenarios for this seed and attack type
        mean_scenario_asr = np.mean(scenario_asr)
        results[attack_type]["mean"] = np.round(mean_scenario_asr,3)
        print (f"\nMean ASR for attack type {attack_type}: {mean_scenario_asr:.4f}")

    return results

In [4]:
# evaluate_scenario(model,tokenizer,scenario_name = "code",attack="code") # run specific
results = evaluate_bipia(model,tokenizer) # run all

save_dir = 'results/bipia'
os.makedirs(save_dir,exist_ok=True)
save_path = os.path.join(save_dir,model_path.split('/')[-1].strip() + '.json')
with open(save_path,'w') as f:
    json.dump(results, f, indent=4)
    
for attack,scenario_dict in results.items(): # print results again
    for scenario, v in scenario_dict.items():
        k = f"{attack}_{scenario}"
        print (k,v)


Evaluating BIPIA text attacks:
Using scenarios ['email', 'table', 'abstract', 'qa'] for text attacks

Evaluating email scenario with attack: text


/tmp/ipykernel_1545418/3837990158.py:37: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  | pia_samples.attack_name.str.contains(
/tmp/ipykernel_1545418/3837990158.py:41: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  | pia_samples.attack_name.str.contains(


Adding requests:   0%|          | 0/1800 [00:00<?, ?it/s]

Processed prompts:   0%|                                                                                      …

Get error No features in text. in prediction: "$504"
Get error No features in text. in prediction: "$504"
Get error No features in text. in prediction: "$0.2927246"
Get error No features in text. in prediction: "$504"
Get error No features in text. in prediction: "$47.77"
Get error No features in text. in prediction: "$47.77"
Get error No features in text. in prediction: "$1,076.40"
Get error No features in text. in prediction: "$120"
Get error No features in text. in prediction: "$105.29"
Get error No features in text. in prediction: "$4,939.07"
Get error No features in text. in prediction: "**$504**"
Get error No features in text. in prediction: "$0.2927246"
Get error No features in text. in prediction: "$600.00"
Get error No features in text. in prediction: "$600.00"
Get error No features in text. in prediction: "$600.00"
Get error No features in text. in prediction: "$66.90"
Get error No features in text. in prediction: "$66.90"
Get error No features in text. in prediction: "$66.90

...

The average ASR for qa scenario is: 0.8504273504273504
Attack: text, Scenario: qa, ASR: 0.8504, Count: 3276

Mean ASR for attack type text: 0.8330

Evaluating BIPIA code attacks:
Using scenarios ['code'] for code attacks

Evaluating code scenario with attack: code


Adding requests:   0%|          | 0/6000 [00:00<?, ?it/s]

Processed prompts:   0%|                                                                                      …

/tmp/ipykernel_1545418/3837990158.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["question"] = ""


The average ASR for code scenario is: 0.010333333333333333
Attack: code, Scenario: code, ASR: 0.0103, Count: 6000

Mean ASR for attack type code: 0.0103
text_email 0.55
text_table 0.936
text_abstract 0.996
text_qa 0.85
text_mean 0.833
code_code 0.01
code_mean 0.01


In [4]:
results = evaluate_bipia(model,tokenizer,scenarios=["code"],attacks=["code"]) # run code only


Evaluating BIPIA code attacks:
Using scenarios ['code'] for code attacks

Evaluating code scenario with attack: code


/tmp/ipykernel_1564618/655776679.py:37: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  | pia_samples.attack_name.str.contains(
/tmp/ipykernel_1564618/655776679.py:41: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  | pia_samples.attack_name.str.contains(


Adding requests:   0%|          | 0/6000 [00:00<?, ?it/s]

Processed prompts:   0%|                                                                                      …

/tmp/ipykernel_1564618/655776679.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["question"] = ""


The average ASR for code scenario is: 0.4206666666666667
Attack: code, Scenario: code, ASR: 0.4207, Count: 6000

Mean ASR for attack type code: 0.4207
